In [1]:

import string
import re
import os
import nltk
import pandas as pd
import numpy as np
import json
import tensorflow as tf
from tensorflow import keras

SEED = 1013
np.random.seed(SEED)

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model
from sklearn.model_selection import StratifiedKFold


# In[ ]:


import pandas as pd 
import numpy as np 
import torch.nn as nn
from pytorch_pretrained_bert import BertTokenizer, BertModel
import torch
from torchnlp.datasets import imdb_dataset
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report


In [2]:

def test(): # feed raw text in BERT so we will not preprocess
    
    sentence_maxlen = 0
    all_favor_tweets = []
    all_against_tweets = []
    
    x_test = []
    y_test = []
    all_favor_tweets_test = []
    all_against_tweets_test = []
    with open(test_data_file, 'r') as testfile:
        for line in testfile:
            line = line.replace('#SemST', '').strip()
            line = line.split('\t')
        

            if line[0] != 'ID' and line[3] == 'FAVOR':
                tweet = line[2]
                #tweet = process_tweet(tweet)
                if len(tweet) > sentence_maxlen:
                    sentence_maxlen = len(tweet)
                all_favor_tweets_test.append(tweet)
            elif line[0].strip() != 'ID' and line[3].strip() == 'AGAINST':
                tweet = line[2]
                #tweet = process_tweet(tweet)
                if len(tweet) > sentence_maxlen:
                    sentence_maxlen = len(tweet)
                all_against_tweets_test.append(tweet)

    x_test = all_favor_tweets_test + all_against_tweets_test
    y_test = np.append(np.ones(len(all_favor_tweets_test)), np.zeros(len(all_against_tweets_test)))

    
    return x_test, y_test, sentence_maxlen

In [3]:
train_data_file = '/home/parush/stance/Experiments/stance_mohammed/train.txt'
test_data_file = '/home/parush/stance/Experiments/stance_mohammed/test.txt'
TARGETS = [ 'Atheism','Climate Change is a Real Concern', 'Feminist Movement','Hillary Clinton', 'Legalization of Abortion' ]

df = pd.read_csv(train_data_file, sep='\t')
df1 = pd.read_csv(test_data_file, sep='\t')


In [66]:
# train_data_file = '/home/parush/stance/Experiments/SomasundaranWiebe-politicalDebates/train.txt'
# test_data_file = '/home/parush/stance/Experiments/SomasundaranWiebe-politicalDebates/test.txt'
# TARGETS = ['god','healthcare','guns','gayRights','abortion', 'creation']
# df = pd.read_csv(train_data_file, sep='\t')
# df1 = pd.read_csv(test_data_file, sep='\t')


In [5]:
# train_data_file = '/home/parush/stance/Experiments/Data_MPCHI/train.txt'
# test_data_file = '/home/parush/stance/Experiments/Data_MPCHI/test.txt'
# TARGETS = ['Are E-Cigarettes safe?','Does MMR Vaccine lead to autism in children?',
#       'Does Sunlight exposure lead to skin cancer?','Does Vitamin C prevent common cold?',
#       'Should women take HRT post-menopause?']
# df = pd.read_csv(train_data_file, sep='\t')
# df1 = pd.read_csv(test_data_file, sep='\t')


In [6]:
test_texts, test_labels, sen_maxlen = test()

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
test_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:511], test_texts))


In [8]:
test_tokens_ids = list(map(tokenizer.convert_tokens_to_ids, test_tokens))
test_tokens_ids = pad_sequences(test_tokens_ids, maxlen=128, truncating="post", padding="post", dtype="int")
test_y = np.array(test_labels) == 1

In [9]:
class BertBinaryClassifier(nn.Module):
    def __init__(self, dropout=0.1):
        super(BertBinaryClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, tokens, masks=None):
        _, pooled_output = self.bert(tokens, attention_mask=masks, output_all_encoded_layers=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        proba = self.sigmoid(linear_output)
        return proba


In [10]:

test_masks = [[float(i > 0) for i in ii] for ii in test_tokens_ids]
test_masks_tensor = torch.tensor(test_masks)
test_tokens_tensor = torch.tensor(test_tokens_ids)
test_y_tensor = torch.tensor(test_y.reshape(-1, 1)).float()

In [11]:
BATCH_SIZE = 32 # you can change it
EPOCHS = 3 # do not train for too many epochs as a large model like this will surely overfit and overflow the ram and disk
learning_rate = 2e-5 # you can decrease it more like 3e-5 but do not increase it more than 1e-5

In [12]:
test_dataset =  torch.utils.data.TensorDataset(test_tokens_tensor, test_masks_tensor, test_y_tensor)
test_sampler =  torch.utils.data.SequentialSampler(test_dataset)
test_dataloader =  torch.utils.data.DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)



In [13]:
bert_clf = torch.load("/home/parush/bert_models/bert_debates")



In [14]:
bert_clf.eval()
bert_predicted = []
all_logits = []
with torch.no_grad():
    for step_num, batch_data in enumerate(test_dataloader):
      
        token_ids, masks, labels = tuple(t for t in batch_data)
        logits = bert_clf(token_ids, masks)
        loss_func = nn.BCELoss()
        loss = loss_func(logits, labels)
        numpy_logits = logits.cpu().detach().numpy()

        bert_predicted += list(numpy_logits[:, 0] > 0.5)
        all_logits += list(numpy_logits[:, 0])


# In[ ]:


print(classification_report(test_y, bert_predicted, digits = 4))



              precision    recall  f1-score   support

       False     0.7659    0.2196    0.3413       715
        True     0.3145    0.8421    0.4580       304

    accuracy                         0.4053      1019
   macro avg     0.5402    0.5308    0.3996      1019
weighted avg     0.6312    0.4053    0.3761      1019

